# Overview and approach
Ok, so i want to get to get following tasks before predictions.
- Hypothesis: These are the major tasks that will guide us in our machine learning process
> Cancalleation_reason is associated with start and destination airports and airline <br>
> Delays are associated with airline and planes ( data of planes is obtained from separate data source ) <br>
> Diversions can be associated with destination and condition( data of conditions isn't available )
- Insights: These are mini-tasks that i want to check 
> All airlines with their cancellation reasons in sorted order <br>
> All airlines with their delay types in sorted order <br>
> All airlines and their diversion <br>
> A ranking function for airlines that considers scores of airline's flight delay, diversion and cancellation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

from pprint import pprint
from prettytable import PrettyTable

### Birdeye view of Flights data

In [2]:
# 469000 rows are very close approximate for one month data
data = pd.read_csv("flights.csv", nrows=469000 ,low_memory=False)
n_rows, n_cols =  data.shape
print "ROWS: ", n_rows, " COLS: ", n_cols
data.head(10)

ROWS:  469000  COLS:  31


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,1,1,4,DL,806,N3730B,SFO,MSP,25,...,610.0,8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
6,2015,1,1,4,NK,612,N635NK,LAS,MSP,25,...,509.0,-17.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
7,2015,1,1,4,US,2013,N584UW,LAX,CLT,30,...,753.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,1,1,4,AA,1112,N3LAAA,SFO,DFW,30,...,532.0,-13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
9,2015,1,1,4,DL,1173,N826DN,LAS,ATL,30,...,656.0,-15.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data.dtypes

YEAR                     int64
MONTH                    int64
DAY                      int64
DAY_OF_WEEK              int64
AIRLINE                 object
FLIGHT_NUMBER            int64
TAIL_NUMBER             object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
SCHEDULED_DEPARTURE      int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME           int64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE                 int64
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL        int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED                 int64
CANCELLED                int64
CANCELLATION_REASON     object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT_DELAY    float64
WEATHER_DELAY          float64
dtype: object

### Let's take a look at delays, cancellation_reasons and diversion counts

In [4]:
print "DIFFENT TPYE OF DELAYS" 
for dt,name in zip( data.dtypes , data.columns):
    if "DELAY" in name:
        print dt , " ",name ,  ", Instances" , data[name].count() # count returns number of non-null values

DIFFENT TPYE OF DELAYS
float64   DEPARTURE_DELAY , Instances 457355
float64   ARRIVAL_DELAY , Instances 456066
float64   AIR_SYSTEM_DELAY , Instances 95828
float64   SECURITY_DELAY , Instances 95828
float64   AIRLINE_DELAY , Instances 95828
float64   LATE_AIRCRAFT_DELAY , Instances 95828
float64   WEATHER_DELAY , Instances 95828


In [5]:
print "DIFFENT TPYE OF CANCELLATION REASONS" #Reason for Cancellation of flight: A - Airline/Carrier; B - Weather; C - National Air System; D - Security
for cancellation_reason in data.CANCELLATION_REASON[data.CANCELLATION_REASON.notnull()].unique():
    if cancellation_reason:
        print cancellation_reason , data[data.CANCELLATION_REASON == cancellation_reason].CANCELLATION_REASON.count()

DIFFENT TPYE OF CANCELLATION REASONS
A 2869
B 7011
C 2087
D 1


In [6]:
print "DIVERTED FLIGHTS", data.DIVERTED.sum()


DIVERTED FLIGHTS 966


# Insights Section
In this section, i will study the relationship of airlines & airports with 3-Bad things( diversion, cancellation and delays ) <br/>
It will provide helpful information for consumers and allow us to explore data patterns.

In [7]:
#LET'S IMPORT THE AIRLINES DATA
airlines = pd.read_csv("airlines.csv")
airlines_list = airlines.IATA_CODE.unique()
airlines_dict = dict()
airlines.head(5)
for item in airlines.iterrows():
    airlines_dict[ item[1][0] ] =item[1][1]
print airlines_dict

{'AA': 'American Airlines Inc.', 'OO': 'Skywest Airlines Inc.', 'DL': 'Delta Air Lines Inc.', 'NK': 'Spirit Air Lines', 'HA': 'Hawaiian Airlines Inc.', 'F9': 'Frontier Airlines Inc.', 'AS': 'Alaska Airlines Inc.', 'US': 'US Airways Inc.', 'B6': 'JetBlue Airways', 'MQ': 'American Eagle Airlines Inc.', 'WN': 'Southwest Airlines Co.', 'VX': 'Virgin America', 'EV': 'Atlantic Southeast Airlines', 'UA': 'United Air Lines Inc.'}


## Relation between Airline and Cancellation_reason
From this data, we can see that some airlines have high cancellation rate  meanwhile some have lower eventhough they have high traffic. <br>

In [8]:
canc_table_rel = []
for airline in airlines_list:
    canc_inst = [airline]
    count_of_airline = len(data[data.AIRLINE == airline  ])
    for reason in ["A","B","C","D"]:
        canc_inst.append(   data[(data.AIRLINE == airline)  & (data.CANCELLATION_REASON == reason) ].CANCELLATION_REASON.count()   )
    canc_inst.append(count_of_airline)
    canc_table_rel.append( canc_inst )
        

In [10]:
#table = pd.crosstab(data.AIRLINE,data.CANCELLATION_REASON)
#table_rel = []
t = PrettyTable(["Airlines - Cancellation_reason", "A","B","C","D","Total Flights(null inc.)"])
for item in canc_table_rel:
    total_flights = data[ data.AIRLINE == item[0] ].AIRLINE.count()
    t.add_row( [item[0], item[1],item[2],item[3],item[4],item[5]] )
    #table_rel.append( ( item[0] , item[1][0] + item[1][1] + item[1][2] ,total_flights ) )
print t


+--------------------------------+-----+------+-----+---+--------------------------+
| Airlines - Cancellation_reason |  A  |  B   |  C  | D | Total Flights(null inc.) |
+--------------------------------+-----+------+-----+---+--------------------------+
|               UA               | 118 | 840  |  8  | 0 |          38331           |
|               AA               | 291 | 595  |  11 | 0 |          43931           |
|               US               | 185 | 701  |  73 | 0 |          33393           |
|               F9               |  62 |  27  |  0  | 0 |           6796           |
|               B6               |  92 | 929  |  80 | 1 |          21538           |
|               OO               | 346 | 687  | 228 | 0 |          48021           |
|               AS               |  29 |  34  |  0  | 0 |          13209           |
|               NK               |  12 |  76  |  10 | 0 |           8689           |
|               WN               | 672 | 1034 |  58 | 0 |        

*Lets go down the rabbit hole and find out which airlines are particularly bad for their cancellation rate. This info. can be helpful for consumers.*

In [13]:
sorted_by_cancellation = sorted(canc_table_rel, key=lambda tup: float(tup[1]+tup[2]+tup[3]+tup[4])/tup[5] , reverse=False)
print "RATING OF AIRLINE BY CANCELLATION( best to worse )"
t = PrettyTable(["Airline", "Cancellations","Total Flights"])
for item in sorted_by_cancellation:
    t.add_row( [airlines_dict[item[0]], item[1]+item[2]+item[3]+item[4],item[5]] )
print t

RATING OF AIRLINE BY CANCELLATION( best to worse )
+------------------------------+---------------+---------------+
|           Airline            | Cancellations | Total Flights |
+------------------------------+---------------+---------------+
|    Hawaiian Airlines Inc.    |       26      |      6425     |
|     Alaska Airlines Inc.     |       63      |     13209     |
|     Delta Air Lines Inc.     |      678      |     64267     |
|       Spirit Air Lines       |       98      |      8689     |
|    Frontier Airlines Inc.    |       89      |      6796     |
|        Virgin America        |       80      |      4727     |
|    Southwest Airlines Co.    |      1764     |     99942     |
|    American Airlines Inc.    |      897      |     43931     |
|    United Air Lines Inc.     |      966      |     38331     |
|    Skywest Airlines Inc.     |      1261     |     48021     |
|       US Airways Inc.        |      959      |     33393     |
| Atlantic Southeast Airlines  |      1

### Relation between Airlines and Delays
In this section, we'll study which airlines are heavily effected by which type of delays.

In [14]:
print "DIFFENT TPYE OF DELAYS" 
t = PrettyTable(["Datatype", "Delay Type","Instances", "Total Delay", "Average Delay Per Flight"])
delay_types = []
for dt,name in zip( data.dtypes , data.columns):
    if "DELAY" in name:
        delay_count = data[name].count()
        delay_sum = data[name].sum()
        
        t.add_row( [dt, name , delay_count ,delay_sum , float( delay_sum )/n_rows] )
        delay_types.append( name )
print t

DIFFENT TPYE OF DELAYS
+----------+---------------------+-----------+-------------+--------------------------+
| Datatype |      Delay Type     | Instances | Total Delay | Average Delay Per Flight |
+----------+---------------------+-----------+-------------+--------------------------+
| float64  |   DEPARTURE_DELAY   |   457355  |  4470337.0  |      9.53163539446       |
| float64  |    ARRIVAL_DELAY    |   456066  |  2659993.0  |      5.67162686567       |
| float64  |   AIR_SYSTEM_DELAY  |   95828   |  1276990.0  |      2.72279317697       |
| float64  |    SECURITY_DELAY   |   95828   |    6663.0   |     0.0142068230277      |
| float64  |    AIRLINE_DELAY    |   95828   |  1706239.0  |      3.63803624733       |
| float64  | LATE_AIRCRAFT_DELAY |   95828   |  2182230.0  |       4.6529424307       |
| float64  |    WEATHER_DELAY    |   95828   |   263015.0  |      0.560799573561      |
+----------+---------------------+-----------+-------------+--------------------------+


*Let's see how different airlines are associated with different type of delays*

In [15]:
# foreach airliness
    # foreach delaytype
        #get counts of that flight
        #get the corresponding counts
print "TABLE OF AIRLINE AND IT'S AVERAGE FLIGHT DELAYS"
dely_table_rel = []
for airline in airlines.IATA_CODE:
    dely_inst = [airline]
    count_of_airline = len(data[data.AIRLINE == airline  ])
    
    for delay in delay_types:
        #print airline, " ", delay ,  data[data.AIRLINE == airline  ][delay].sum()
        dely_inst.append(  round( data[data.AIRLINE == airline  ][delay].sum() /count_of_airline,3  ))
    dely_inst.append(count_of_airline)
    dely_table_rel.append( dely_inst )
    
# printing table
t = PrettyTable(["Airline", "DEPARTURE","ARRIVAL", "AIR_SYSTEM", "SECURITY","AIRLINE","LATE_AIRCRAFT","WEATHER"])
for item in dely_table_rel:
        t.add_row( [airlines_dict[item[0]], item[1],item[2],item[3] ,item[4] ,item[5],item[6],item[7]] )
print t

TABLE OF AIRLINE AND IT'S AVERAGE FLIGHT DELAYS
+------------------------------+-----------+---------+------------+----------+---------+---------------+---------+
|           Airline            | DEPARTURE | ARRIVAL | AIR_SYSTEM | SECURITY | AIRLINE | LATE_AIRCRAFT | WEATHER |
+------------------------------+-----------+---------+------------+----------+---------+---------------+---------+
|    United Air Lines Inc.     |   13.682  |  6.208  |   2.974    |  0.001   |  4.526  |     4.364     |  0.863  |
|    American Airlines Inc.    |   10.407  |  6.838  |   2.286    |  0.014   |  4.634  |     5.139     |  0.565  |
|       US Airways Inc.        |   5.058   |  3.023  |   2.909    |   0.03   |  3.013  |     2.424     |   0.23  |
|    Frontier Airlines Inc.    |   17.862  |  18.186 |   7.103    |   0.0    |  4.894  |     9.803     |   0.19  |
|       JetBlue Airways        |   9.542   |  6.978  |   3.744    |  0.052   |  3.536  |     5.022     |  0.436  |
|    Skywest Airlines Inc.     |

In [16]:
sorted_by_delays = sorted(dely_table_rel, key=lambda tup: float(tup[1]+tup[2]+tup[3]+tup[4]+tup[5]+tup[6]+tup[7]) )
print "RATING OF BEST AIRLINE BY DELAYS( best to worse )"
t = PrettyTable(["Airline", "Average of All Delays( Per Flight)","Total Flights"])
for item in sorted_by_delays:
    t.add_row( [airlines_dict[item[0]],  item[1]+item[2]+item[3]+item[4]+item[5]+item[6]+item[7],item[8] ] )
print t

RATING OF BEST AIRLINE BY DELAYS( best to worse )
+------------------------------+------------------------------------+---------------+
|           Airline            | Average of All Delays( Per Flight) | Total Flights |
+------------------------------+------------------------------------+---------------+
|     Alaska Airlines Inc.     |               10.351               |     13209     |
|    Hawaiian Airlines Inc.    |               10.357               |      6425     |
|     Delta Air Lines Inc.     |               11.002               |     64267     |
|       US Airways Inc.        |               16.687               |     33393     |
|        Virgin America        |               16.912               |      4727     |
|    Southwest Airlines Co.    |               21.778               |     99942     |
|       JetBlue Airways        |               29.31                |     21538     |
|    American Airlines Inc.    |               29.883               |     43931     |
| At

### Airlines and flight diversion
Even though flight diversions are normally caused by circumstances like security and weather etc. But lets see if some airlines have particulary higher rate of diversion.

In [17]:
print "Total diverted flights: ",len(data[data.DIVERTED == 1 ])


Total diverted flights:  966


In [18]:
diversion_table_rel = []
for airline in airlines_list:
    div_inst = [airline]
    count_of_airline = len(data[data.AIRLINE == airline  ])

    div_inst.append(   len(data[(data.AIRLINE == airline)  & (data.DIVERTED == 1) ]   ))

    div_inst.append(count_of_airline)
    diversion_table_rel.append( div_inst )

In [19]:
sorted_by_diversion = sorted(diversion_table_rel, key=lambda tup: float(tup[1]) /tup[2], reverse=False)
print "RATING OF BEST AIRLINE BY DIVERSIONS( best to worse )"
t = PrettyTable(["Airline", "Diversions","Total Flights"])
for item in sorted_by_diversion:
    t.add_row( [airlines_dict[item[0]],  item[1],item[2] ] )
print t

RATING OF BEST AIRLINE BY DIVERSIONS( best to worse )
+------------------------------+------------+---------------+
|           Airline            | Diversions | Total Flights |
+------------------------------+------------+---------------+
|    Frontier Airlines Inc.    |     5      |      6796     |
|        Virgin America        |     4      |      4727     |
|    Hawaiian Airlines Inc.    |     6      |      6425     |
|     Delta Air Lines Inc.     |     67     |     64267     |
|       US Airways Inc.        |     45     |     33393     |
| American Eagle Airlines Inc. |     44     |     29856     |
|       Spirit Air Lines       |     13     |      8689     |
|    United Air Lines Inc.     |     65     |     38331     |
|       JetBlue Airways        |     39     |     21538     |
|    American Airlines Inc.    |     85     |     43931     |
|    Southwest Airlines Co.    |    215     |     99942     |
| Atlantic Southeast Airlines  |    139     |     49875     |
|     Alaska Air

### Overall Relative Ranking of an airline
Okay, so now we want to combine all the information above and create a ranking function.<br>
So we are dividing airlines into three sections( Best, Good, Bad ) for each metric. <br>
Best results airlines will be given 3 scores, good will be given 2 and bad will be given 1.


In [20]:
first_block = np.percentile( range(0,14),35)
second_block = np.percentile( range(0,14),65)
third_block = np.percentile( range(0,14),100)

In [21]:
scores_table = []
for i,airline in enumerate( airlines_list):
    # Scores for diversion
    # 3-best, 2-good, 1-bad
    airline_score = [airline]
    count_of_flights = 0
    for j,item in enumerate( sorted_by_diversion):
        if item[0] == airline:
            if j >= 0.0 and j < first_block:
                airline_score.append(3)
            elif j >= first_block and j < second_block:
                airline_score.append(2)
            else:
                airline_score.append(1)
            count_of_flights = item[2]

        
    # Scores for cancellation           
    for j,item in enumerate( sorted_by_cancellation):
        if item[0] == airline:
            if j >= 0.0 and j < first_block:
                airline_score.append(3)
            elif j >= first_block and j < second_block:
                airline_score.append(2)
            else:
                airline_score.append(1)
        
    # Scores for delay
    for j,item in enumerate( sorted_by_delays):
        if item[0] == airline:
            if j >= 0.0 and j < first_block:
                airline_score.append(3)
            elif j >= first_block and j < second_block:
                airline_score.append(2)
            else:
                airline_score.append(1)
    airline_score.append(count_of_flights)
    scores_table.append(airline_score)



In [22]:
# sorting by rating
sorted_score_table = sorted(scores_table, key=lambda tup: tup[1]+tup[2]+tup[3], reverse=True)
print "OVERALL RANKING OF AIRLINE( best to worse )"
t = PrettyTable(["Airline", "Diversion Score","Cancellation Score","Delay Score", "Total Score","Total Flights"])
for item in sorted_score_table:
    t.add_row( [airlines_dict[item[0]],  item[1],item[2],item[3], item[1]+item[2]+item[3],item[4]] )
print t

OVERALL RANKING OF AIRLINE( best to worse )
+------------------------------+-----------------+--------------------+-------------+-------------+---------------+
|           Airline            | Diversion Score | Cancellation Score | Delay Score | Total Score | Total Flights |
+------------------------------+-----------------+--------------------+-------------+-------------+---------------+
|     Delta Air Lines Inc.     |        3        |         3          |      3      |      9      |     64267     |
|    Hawaiian Airlines Inc.    |        3        |         3          |      3      |      9      |      6425     |
|        Virgin America        |        3        |         2          |      3      |      8      |      4727     |
|       US Airways Inc.        |        3        |         1          |      3      |      7      |     33393     |
|    Frontier Airlines Inc.    |        3        |         3          |      1      |      7      |      6796     |
|     Alaska Airlines Inc.  

** Conclusion: We can see that some airlines are clearly dominating other lines when it comes to quality service in terms of flight delays, diversion and cancellations.<br>
Also some airlines are really good in one aspect but are struggling in other aspects<br>
Number of flights handled by airline is also being a hurdle in service**